In [1]:
import csv
import pandas as pd
from datetime import datetime
import time
import numpy as np
import scipy.stats as st
from scipy.special import comb, perm
from pathlib import Path
import os
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from itertools import combinations
from igraph import *
import json

In [2]:
year = '2021'

In [3]:
root_path = Path(fr'/Volumes/SP PHD U3/{year}_PTT_raw_data/covid-19/iii_not_merged')
root_path_csv = Path(fr'/Volumes/SP PHD U3/{year}_PTT_raw_data/covid-19_csv')

In [12]:
for file in root_path.glob(f'*{year}*'):
    if '._' not in file.stem:
        print(file.stem)

202105-type1-or2
202106-type1-or2
202102-type1-or2
202101-type1-or2
202103-type1-or2
202104-type1-or2


In [13]:
def count_time_gap(time_a, time_c):
    time_format = '%Y/%m/%d %H:%M:%S'
    timeString_a = time_a
    struct_time_a = time.strptime(timeString_a, time_format)
    time_stamp_a = int(time.mktime(struct_time_a))
    timeString_c = time_c
    struct_time_c = time.strptime(timeString_c, time_format)
    time_stamp_c = int(time.mktime(struct_time_c))
    time_gap = time_stamp_c - time_stamp_a
    return time_gap

In [14]:
file.stem.split('-')[0]

'202104'

In [15]:
'iii_'+file.stem.split('-')[0]+'_Gossiping_all'

'iii_202104_Gossiping_all'

# 轉CSV檔

In [17]:
#清理後的留言時間為time，原始為post_time
year = 2021

for file in root_path.glob(f'*{year}06*'):
    if '._' not in file.stem: #去除隱藏檔
        print(f'opening {file.stem} at {time.ctime()}')
        df_file_name = 'iii_'+file.stem.split('-')[0]+'_Gossiping_all'

        author_reply = []  
        header = ['id', 'time', 'title', 'author_id', 
                  'from_id', 'reaction', 'post_time', 'date_a', 'date_c'] 
        with open(root_path_csv / f'{df_file_name}.csv', 'w', newline='') as written_file:
            writeCsv = csv.writer(written_file, delimiter=',')
            writeCsv.writerow(header)

        with open(file , 'r', encoding='utf8') as reader:
            jf = json.loads(reader.read())

            for idx, article in enumerate(jf):
                if article['id'].startswith('Gossiping'):
    
                    for comment in article['comments']:
                        time_a = article['time']
                        time_c = comment['time']
                        date_a = time_a.split(' ')[0]
                        date_c = time_c.split(' ')[0]

                        time_gap = count_time_gap(time_a, time_c)

                        if 0 > time_gap >= -60 and type(article['id'])==str:                            
                            author_reply.append((
                                article['id'], #id
                                time_a, #time
                                article['title'], #title
                                article['author_id'], #author_id
                                comment['from_id'], 
                                comment['reaction'], 
                                time_a, #[post_time]
                                date_a,
                                date_c
                            ))

                        if time_gap >= 0 and type(article['id'])==str:                            
                            author_reply.append((
                                article['id'], #id
                                time_a, #time
                                article['title'], #title
                                article['author_id'], #author_id
                                comment['from_id'], 
                                comment['reaction'], 
                                time_c, #[post_time]
                                date_a,
                                date_c
                            ))

                if idx % 10000 == 0:
                    print(f'saving {idx} at {time.ctime()}')
                    with open(root_path_csv/f'{df_file_name}.csv', 'a', newline='') as written_file:
                        writeCsv = csv.writer(written_file, delimiter=',')
                        writeCsv.writerows(author_reply)                       
                    author_reply = []      

            if idx % 10000 != 0:
                print(f'saving {idx} at {time.ctime()}')
                with open(root_path_csv/f'{df_file_name}.csv', 'a', newline='') as written_file:
                    writeCsv = csv.writer(written_file, delimiter=',')
                    writeCsv.writerows(author_reply)   

print('done')

opening 202106-type1-or2 at Sun Jul  4 18:02:22 2021
saving 0 at Sun Jul  4 18:03:24 2021
saving 10000 at Sun Jul  4 18:03:37 2021
saving 20000 at Sun Jul  4 18:03:51 2021
saving 30000 at Sun Jul  4 18:04:03 2021
saving 40000 at Sun Jul  4 18:04:16 2021
saving 50000 at Sun Jul  4 18:04:29 2021
saving 60000 at Sun Jul  4 18:04:41 2021
saving 70000 at Sun Jul  4 18:04:54 2021
saving 80000 at Sun Jul  4 18:05:06 2021
saving 81578 at Sun Jul  4 18:05:09 2021
done


# 標題分類

In [18]:
def cat_filter():
    for rows in title_df.itertuples():        
            
        if 'Re:' in rows.title:
            title_df.loc[rows.Index, 'category'] = 'Re'
            
        else:
            if '[爆卦]' in rows.title:
                title_df.loc[rows.Index, 'category'] = '爆卦'
            elif '[問卦]' in rows.title:
                title_df.loc[rows.Index, 'category'] = '問卦'
            elif '[新聞]' in rows.title:
                title_df.loc[rows.Index, 'category'] = '新聞'
            else:
                title_df.loc[rows.Index, 'category'] = 'others'
        
    return title_df

In [60]:
# cat = ['[請問]', '[爆卦]', '[問卦]', '[新聞]', '[舊聞]', '[問題]', '[分享]', '[活動]', '[參選]', 
#        '[問話]', '[廢文]', '[心得]', '[協尋]', '[公告]', '[尋人]', '[地震]', '[請益]', '[無]']

In [20]:
for file in root_path_csv.glob(f'*{year}06*Gossiping_all*'):
    if '._' not in file.stem:
        print(f'opening {file.stem} at {time.ctime()}')     
        raw_data = pd.read_csv(file)
        print(raw_data.shape)
        raw_data = raw_data.dropna(axis=0, how='any')
        
        title_df = raw_data[['id', 'title']].drop_duplicates()
        title_df = cat_filter()          
        new_data = title_df.merge(raw_data)
        
        print(f'saving {file.stem} at {time.ctime()}')
        print(new_data.shape)
        new_data.to_csv(root_path_csv/f'{file.stem}.csv', index=False)

        print('='*100)

opening iii_202106_Gossiping_all at Sun Jul  4 18:06:00 2021
(4517417, 9)
saving iii_202106_Gossiping_all at Sun Jul  4 18:06:18 2021
(4517417, 10)


# covid-19相關文章

In [21]:
def topic_filter(word_topic, topic):
    for rows in title_df.itertuples():        
        for word in word_topic:
            
            if word in rows.title:
                title_df.loc[rows.Index, topic] = 1
    
    return title_df

In [22]:
word_covid = ['變種', '四級', '疾管家', '封城', '復星', '萬華', '醫院', '方艙', '院內', '陽性率', '篩檢', '復必泰',
              '超前部署', '醫護', '輕症', 'J&J', '抗疫', '茶室', '衛福部', '指揮中心', '確診', '莫德納', '南送', 
              '輝瑞', '三級', 'PCR', '境外', '重症', '肺炎', '採檢', 'BioNTech', '高端', 'Moderna', '國藥', 
              '諾富特', '疫情', '快篩', '校正回歸', '防疫', '疫調', '偽陰', '機師', '實聯制', '本土', '傳播鏈', '疫苗',
              'BNT', '超前', 'moderna', '科興', '陳時中', '聯亞', '3+11', '病毒株', '口罩', '廣篩', '國產疫苗', 
              '染疫', '隔離', '普篩', '實名制', 'AZ', 'Ct值', '感染', '群聚', '偽陽', '人與人', '社區', '嬌生']

In [23]:
word_vaccine = ['疫苗', '國產疫苗', '聯亞','高端',
                'AZ', 'Moderna', 'moderna', '莫德納', '嬌生', 'J&J', 'BNT', 'BioNTech', '復必泰', 
                '科興', '國藥', '復星',
                '輝瑞']

In [24]:
word_cecc = ['衛福部', '指揮中心', '陳時中', '疫調', '校正回歸', '疾管家', '實聯制', '實名制',  
             '超前', '超前部署', '南送']

In [28]:
# data_range = 'covid'
# word_lst = word_covid
data_range = 'vaccine'
word_lst = word_vaccine
# data_range = 'CECC'
# word_lst = word_cecc

for file in root_path_csv.glob(f'*{year}06*Gossiping_all*'):
    if '._' not in file.stem: #去除隱藏檔
        print(f'opening {file.stem} at {time.ctime()}')
        
        raw_data = pd.read_csv(file)

        title_df = raw_data[['id', 'title']].drop_duplicates()
        print(f'num of article_all: {len(set(raw_data.id))}')
        
        df_file_name = str(file.stem).replace('all', data_range)
        title_df = topic_filter(word_lst, data_range)
        covid_data = title_df[title_df[data_range] == 1]
#         print(len(set(covid_data.id)))
        
        new_data = covid_data.merge(raw_data, how='left')
        print(f'num of article_covid: {len(set(new_data.id))}')
        
        print(f'saving {file.stem} at {time.ctime()}')
        new_data.to_csv(root_path_csv/f'{df_file_name}.csv', index=False)
        
        print('='*100)
                      
print('done')

opening iii_202106_Gossiping_all at Sun Jul  4 18:09:10 2021
num of article_all: 80833
num of article_covid: 16153
saving iii_202106_Gossiping_all at Sun Jul  4 18:09:26 2021
done


# 母體計算

In [12]:
df = pd.DataFrame()
data_range = 'covid'
num_comment = 0

for file in root_path_csv.glob(f'*{year}*Gossiping*{data_range}*'):
    if '._' not in file.stem:
        print(file.stem, time.ctime())
        month_df = pd.read_csv(file)
        num_comment = num_comment + len(month_df)
        
        month_need_df = month_df[['id', 'author_id', 'from_id']]
            
        df = pd.concat([df, month_need_df])
        
print('done') 
print(num_comment)

iii_202102_Gossiping_covid Wed Nov 10 20:11:56 2021
iii_202101_Gossiping_covid Wed Nov 10 20:11:57 2021
iii_202103_Gossiping_covid Wed Nov 10 20:11:57 2021
iii_202104_Gossiping_covid Wed Nov 10 20:11:58 2021
iii_202105_Gossiping_covid Wed Nov 10 20:11:58 2021
iii_202106_Gossiping_covid Wed Nov 10 20:12:02 2021
done
4196595


In [13]:
4196595/16229533

0.25857768057774677

In [8]:
len(df)

16229533

In [5]:
len(set(df['id']))

369545

In [37]:
len(set(df['author_id']))

9204

In [38]:
len(set(df['from_id']))

78527

In [10]:
len(set(df['from_id']).union(set(df['author_id'])))

172183